# Agent with tools
This notebook illustrates how to build a simple agent capable of using tools. 
We'll use crewAI to illustrate how agentics implements logical transduction

## Initialize environment


In [13]:
! uv pip install agentics-py

import os
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)


if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    load_dotenv("/content/drive/MyDrive/.env")
    from google.colab import drive

    drive.mount("/content/drive")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

Using Python 3.12.11 environment at: /Users/gliozzo/miniforge3/envs/agentics
Audited 1 package in 27ms
In Colab: False


## Create a new tool
Wrap the Duck Duck Go Search API call into a CrewAI tool using the @tool decorator

In [14]:
from agentics.core.llm_connections import get_llm_provider
from crewai.tools import tool
from ddgs import DDGS

## Define a Crew AI tool to get news for a given date using the DDGS search engine
@tool("fetch_data_async")
def web_search(query: str) -> str:
    """ Fetch web search results for the given query using DDGS."""
    return str(DDGS().text(query, max_results=10))    

## Define an Agent

Use crew AI agent directly create a document searcher agent, who is instructed to use the web_search tool defined above. Note that reasoning steps and other parameters can be customized. 


In [15]:
from crewai import Agent
doc_agent = Agent(
    role="Doc Searcher",
    goal="Find answers to questions from the user using the available web search tool.",
    backstory="A helpful assistant for extensive web search reports.",
    tools=[web_search],
    reasoning=True,
    reasoning_steps=2,
    verbose=True,
    llm=get_llm_provider()
)

2025-09-05 15:54:35.910 | DEBUG    | agentics.core.llm_connections:get_llm_provider:29 - No LLM provider specified. Using the first available provider.
2025-09-05 15:54:35.911 | DEBUG    | agentics.core.llm_connections:get_llm_provider:31 - Available LLM providers: ['watsonx', 'gemini', 'openai']. Using 'watsonx'


## Define a Task

In [16]:
from crewai import Task

doc_task = Task(
    description="""Your task is to perform an extensive web search about
    the following question {question} and return a document providing answers to 
    the questions that explore several interesting aspects, each of them supported 
    by pertinent information from web search.  """,
    expected_output="""A markdown document with introduction and different sections""",
    agent=doc_agent,
    markdown=True
)

## Initialize a Crew

In [17]:
from crewai import Crew

crew = Crew(
    agents=[doc_agent],
    tasks=[doc_task],
    verbose=True,
)

## Kickoff the crew

Answer the question on the bar above. In the Agent Trace stack you can see the plan made by the agent and the different execution steps which might involve tool calls.

In [19]:
result = crew.kickoff(inputs={"question": input("Ask your question> ") })

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 47bc0326-90fc-4ca1-b0d6-fa379b050fa2                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Users/gliozzo/miniforge3/envs/agentics/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/Users/gliozzo/miniforge3/envs/agentics/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭─────────────────────────────────────────────── 🧠 Reasoning Plan ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ## Task Understanding and Plan                                                                                 │
│  As a Doc Searcher, my primary goal is to find answers to questions from the user using the available web       │
│  search tool. The assigned task involves performing an extensive web search about the question "Where is        │
│  Catania?" and returning a document that provides answers to questions exploring several interesting aspects    │
│  of Catania, supported by pertinent information from the web search.                                            │
│                                                                                                                 │
│  ### 1. Understanding of the Task                                                                               │
│  From my professional perspective, this task requires a comprehensive search of the web to gather information   │
│  about Catania, including its geographical location, historical significance, cultural aspects, and any other   │
│  interesting facts. The task demands the ability to navigate through various web pages, identify relevant       │
│  information, and compile it into a structured document.                                                        │
│                                                                                                                 │
│  ### 2. Key Steps to Complete the Task                                                                          │
│  To complete this task, I will take the following key steps:                                                    │
│  - **Step 1: Initial Search** - Use the fetch_data_async tool to perform an initial web search about Catania    │
│  to gather basic information such as its location, country, and any notable features.                           │
│  - **Step 2: In-Depth Search** - Based on the initial findings, conduct more specific searches to explore       │
│  different aspects of Catania, such as its history, culture, economy, and tourist attractions.                  │
│  - **Step 3: Information Compilation** - Compile the information gathered into a structured markdown document   │
│  with sections dedicated to different aspects of Catania.                                                       │
│  - **Step 4: Verification and Validation** - Verify the accuracy of the information compiled and validate the   │
│  relevance of the content to ensure it meets the task's requirements.                                           │
│                                                                                                                 │
│  ### 3. Approaching Challenges                                                                                  │
│  Challenges might arise in terms of information accuracy, relevance, and the sheer volume of data available on  │
│  the web. To overcome these challenges, I will:                                                                 │
│  - Use specific keywords and filters to refine the search results and ensure relevance.                         │
│  - Cross-verify information across multiple sources to ensure accuracy.                                         │
│  - Employ my expertise in web searching to efficiently navigate through the vast amount of data available.      │
│                                                                                                                 │
│  ### 4. Strategic Use of Available Tools                                                                        │
│  The fetch_data_async tool will be strategically used for:                                                      │
│  - **Initial and In-Depth Searches**: To fetch data fro

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Doc Searcher                                                                                            │
│                                                                                                                 │
│  Task: Your task is to perform an extensive web search about                                                    │
│      the following question Where is Catania? and return a document providing answers to                        │
│      the questions that explore several interesting aspects, each of them supported                             │
│      by pertinent information from web search.                                                                  │
│                                                                                                                 │
│  Reasoning Plan:                                                                                                │
│  ## Task Understanding and Plan                                                                                 │
│  As a Doc Searcher, my primary goal is to find answers to questions from the user using the available web       │
│  search tool. The assigned task involves performing an extensive web search about the question "Where is        │
│  Catania?" and returning a document that provides answers to questions exploring several interesting aspects    │
│  of Catania, supported by pertinent information from the web search.                                            │
│                                                                                                                 │
│  ### 1. Understanding of the Task                                                                               │
│  From my professional perspective, this task requires a comprehensive search of the web to gather information   │
│  about Catania, including its geographical location, historical significance, cultural aspects, and any other   │
│  interesting facts. The task demands the ability to navigate through various web pages, identify relevant       │
│  information, and compile it into a structured document.                                                        │
│                                                                                                                 │
│  ### 2. Key Steps to Complete the Task                                                                          │
│  To complete this task, I will take the following key steps:                                                    │
│  - **Step 1: Initial Search** - Use the fetch_data_async tool to perform an initial web search about Catania    │
│  to gather basic information such as its location, country, and any notable features.                           │
│  - **Step 2: In-Depth Search** - Based on the initial findings, conduct more specific searches to explore       │
│  different aspects of Catania, such as its history, culture, economy, and tourist attractions.                  │
│  - **Step 3: Information Compilation** - Compile the information gathered into a structured markdown document   │
│  with sections dedicated to different aspects of Catania.                                                       │
│  - **Step 4: Verification and Validation** - Verify the accuracy of the information compiled and validate the   │
│  relevance of the content to ensure it meets the task's requirements.                                           │
│                                                                                                                 │
│  ### 3. Approaching Challenges                                                                                  │
│  Challenges might arise in terms of information accurac

...

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"Catania history\"}"                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  [{'title': 'Catania - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/Catania', 'body': 'Despite being the  │
│  second city of the island, Catania is the center of the most densely populated Sicilian conurbation, [8]       │
│  which is among the largest in Italy. It has important road and rail …'}, {'title': 'Catania - Visit Sicily',   │
│  'href': 'https://www.visitsicily.info/en/localita/catania/', 'body': 'Catania is a multicoloured universe,     │
│  from the glittering whiteness of its Baroque stuccoes (a UNESCO World Heritage Site) to the lava black of the  │
│  Liotru, the elephant sculpted from basalt …'}, {'title': '17+ Best Things to Do in Catania , Italy (2025       │
│  Sicily Travel Guide)', 'href': 'https://www.savoringitaly.com/catania-italy/', 'body': 'Feb 7, 2025 · Explore  │
│  Catania, Italy, a vibrant Sicilian city under Mount Etna. Discover history, culture, and amazing food with     │
│  our 2025 travel guide.'}, {'title': '10 Best Things to do in Catania , Sicily (2025 Travel Guide)', 'href':    │
│  'https://www.saltinourhair.com/italy/catania-sicily/', 'body': 'Dec 30, 2023 · From vibrant markets to         │
│  beautiful architecture and always in the shadow of Mount Etna, discover all the best things to do in Catania,  │
│  Sicily.'}, {'title': 'Catania travel - Lonely Planet | Sicily, Italy, Europe', 'href':                         │
│  'https://www.lonelyplanet.com/destinations/italy/sicily/catania', 'body': "Discover Baroque gems, lively       │
│  markets and Mt Etna hikes in our Catania travel guide. Plan your trip to Sicily's volcanic seaside city with   │
│  expert tips and tricks."}, {'title': 'Catania | Sicily, Italy, Map, Volcano, & History | Britannica', 'href':  │
│  'https://www.britannica.com/place/Catania', 'body': 'Catania, city, eastern Sicily, Italy, in the broad plain  │
│  of Catania on the Ionian seacoast, south of Mount Etna. The city was founded in 729 bce by Chalcidians         │
│  (settlers from Chalcis in the …'}, {'title': 'Visit Catania : Things to do & Markets - Italia.it', 'href':     │
│  'https://www.italia.it/en/sicily/catania', 'body': "Don't miss Catania, close to M...                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Doc Searcher                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Introduction to Catania                                                                                      │
│  Catania is a city located on the eastern coast of Sicily, Italy, at the foot of Mount Etna, the largest        │
│  volcano in Europe. It is the second-largest city in Sicily and has a rich history, vibrant culture, and        │
│  stunning architecture.                                                                                         │
│                                                                                                                 │
│  ## Geographical Location                                                                                       │
│  Catania is situated in the broad plain of Catania on the Ionian seacoast, south of Mount Etna. The city has a  │
│  population of around 310,000 people and is a major transport hub, economic center, and university city.        │
│                                                                                                                 │
│  ## History of Catania                                                                                          │
│  Catania was founded in 729 BCE by Chalcidians from Naxos, a Greek island. The city has a long and complex      │
│  history, with various dominations and natural disasters shaping its development. Despite this, Catania still   │
│  retains traces of its origins, including its Roman Theatre, which was built on the site of a previous Greek    │
│  theatre in the 2nd century AD.                                                                                 │
│                                                                                                                 │
│  ## Cultural Aspects of Catania                                                                                 │
│  Catania is known for its Baroque architecture, which is a result of the city's rebuilding after the 1693       │
│  earthquake. The city's historic center is a UNESCO World Heritage Site and features many beautiful buildings,  │
│  including the Cathedral of Sant'Agata. Catania is also famous for its vibrant markets, where visitors can      │
│  find fresh produce, seafood, and local products.                                                               │
│                                                                                                                 │
│  ## Tourist Attractions in Catania                                                                              │
│  Catania offers a wide range of tourist attractions, including:                                                 │
│  * The Cathedral of Sant'Agata                                                                                  │
│  * The Roman Theatre                                                                                            │
│  * The Castello Ursino                                                                                          │
│  * The Museo Civico                                                                                             │
│  * The Mercato di Pesce (Fish Market)                                                                           │
│  * The Mount Etna volcano                                                                                       │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 79fac8dc-dc71-4053-8f40-7eb5d69a5933                                                                     │
│  Agent: Doc Searcher                                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 47bc0326-90fc-4ca1-b0d6-fa379b050fa2                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Introduction to Catania                                                                        │
│  Catania is a city located on the eastern coast of Sicily, Italy, at the foot of Mount Etna, the largest        │
│  volcano in Europe. It is the second-largest city in Sicily and has a rich history, vibrant culture, and        │
│  stunning architecture.                                                                                         │
│                                                                                                                 │
│  ## Geographical Location                                                                                       │
│  Catania is situated in the broad plain of Catania on the Ionian seacoast, south of Mount Etna. The city has a  │
│  population of around 310,000 people and is a major transport hub, economic center, and university city.        │
│                                                                                                                 │
│  ## History of Catania                                                                                          │
│  Catania was founded in 729 BCE by Chalcidians from Naxos, a Greek island. The city has a long and complex      │
│  history, with various dominations and natural disasters shaping its development. Despite this, Catania still   │
│  retains traces of its origins, including its Roman Theatre, which was built on the site of a previous Greek    │
│  theatre in the 2nd century AD.                                                                                 │
│                                                                                                                 │
│  ## Cultural Aspects of Catania                                                                                 │
│  Catania is known for its Baroque architecture, which is a result of the city's rebuilding after the 1693       │
│  earthquake. The city's historic center is a UNESCO World Heritage Site and features many beautiful buildings,  │
│  including the Cathedral of Sant'Agata. Catania is also famous for its vibrant markets, where visitors can      │
│  find fresh produce, seafood, and local products.                                                               │
│                                                                                                                 │
│  ## Tourist Attractions in Catania                                                                              │
│  Catania offers a wide range of tourist attractions, including:                                                 │
│  * The Cathedral of Sant'Agata                                                                                  │
│  * The Roman Theatre                                                                                            │
│  * The Castello Ursino                                                                                          │
│  * The Museo Civico                                                                                             │
│  * The Mercato di Pesce (Fish Market)                                                                           │
│  * The Mount Etna volcano                             

## Display the results as markdown